In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import sqlite3
import time
import pandas as pd

In [2]:
!dir .\chromedriver.exe

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FAE3-2A09

 C:\Develops\teamproject2 디렉터리

2021-07-28  오후 12:02        11,107,328 chromedriver.exe
               1개 파일          11,107,328 바이트
               0개 디렉터리  46,299,541,504 바이트 남음


In [3]:
browser = webdriver.Chrome('./chromedriver.exe')

In [7]:
# news = []
conn = sqlite3.connect('./db_scrap.stock')
c = conn.cursor()
# c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, press TEXT , stock TEXT, posi_nega TEXT)")
c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, content TEXT, press TEXT , stock TEXT, posi_nega TEXT)")

for num in range(601,801):
    browser.get('https://finance.naver.com/news/market_special.nhn?&page='+str(num))
    html = browser.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    for i in range(0,20):
        html = browser.page_source
        soup = BeautifulSoup(html,'html.parser')
        
        names = soup.select('td.publicSubject')
        name = names[int(i)]
        title = name.select('a')[0].text.strip()
        href = ('https://finance.naver.com' + name.select('a')[0]['href'])

        datetimes = soup.select('td.wdate')
        datetime = datetimes[int(i)]
        time = datetime.text.strip()[-5:]

        medias = soup.select('td.press')
        media = medias[int(i)]
        magazine = media.text.strip()
        
        browser.get(href)
        
        try:
            WebDriverWait(browser, 1).until(EC.alert_is_present())
            alert = browser.switch_to.alert     
            alert.dismiss()
            browser.back()
        
        except: 
            data_html = browser.page_source
            data_soup = BeautifulSoup(data_html,'html.parser')

            for tag in data_soup.find_all(['li', 'h3']):
                tag.replace_with('')

            date = data_soup.select('span.article_date')[0].text.strip()
            data = data_soup.select('div.articleCont')[0].text.strip()

#             news.append([date, time, magazine, data, title])
#             c.execute("INSERT INTO article( date , time , press , title, stock , posi_nega ) VALUES(?,?,?,?,?,?,?)",( date, time, magazine, title,'stock','posi_nega'))
            c.execute("INSERT INTO article( date , time , press , title, content, stock , posi_nega ) VALUES(?,?,?,?,?,?,?)",( date, time, magazine, title, data,'stock','posi_nega'))

            browser.back()
    
conn.commit()
c.close()
# news

In [145]:
# pd_data = pd.DataFrame(news, columns=['date','time','press','content','title'])
# pd_data

,date,time,press,content,title
0,2021-08-11,15:36,연합뉴스,[그래픽] 최근 남북관계 주요 일지(서울=연합뉴스) 이재윤 기자 = 김영철 노동당 ...,북한 한미연합훈련 맹비난에 경협주 하락(종합)
1,2021-08-11,14:38,연합뉴스,원티드랩[프레인키웨스트 제공. 재판매 및 DB금지](서울=연합뉴스) 이미령 기자 =...,"원티드랩, 코스닥 상장 첫날 '따상' 성공(종합)"
2,2021-08-11,14:37,헤럴드경제,9월 1일부터 .com 도메인 7% 인상향후 8년에 걸쳐 인상 예정[가비아 제공][...,가비아 COM 도메인 다년 연장 이벤트
3,2021-08-11,14:19,아시아경제,한솔로지스틱스가 강세다. 2분기 실적 개선 소식이 주가에 영향을 준 것으로 보인다....,"한솔로지스틱스, 해상운임 급등에 순익 8배…재평가 기대"
4,2021-08-11,14:05,파이낸셜뉴스,[파이낸셜뉴스]코로나19 4차 대유행 속 게임체인저로 꼽히는 머크의 코로나19 치료...,"엔지켐생명과학, 경구형 코로나 치료제 호주서 상용화 첫발 소식에.."
5,2021-08-11,13:52,이데일리,[이데일리 이은정 기자] KT(030200)가 2분기 ‘어닝 서프라이즈’에 이어 장...,"KT, 장기 실적성장·긍정적 배당 전망에 '강세'"
6,2021-08-11,11:23,이데일리,[이데일리 이은정 기자] SK바이오팜(326030)이 뇌전증 치료제 ‘엑스코프리’(...,"SK바이오팜, 엑스코프리·수노시 매출 호조 전망에 '강세'"
7,2021-08-11,10:47,아시아경제,바이오로그디바이스가 강세다. 백신 접종 선도국조차 델타 변이 영향으로 코로나19 사...,"바이오로그, 확산 또 확산 델타 변이…99.9% 코로나19 예방.."
8,2021-08-11,10:45,아시아경제,루트로닉이 올해 2분기 흑자 전환했다는 소식에 급등세다.11일 오전 10시45분 기...,"루트로닉, 2Q 흑자 전환에 급등"
9,2021-08-11,10:43,파이낸셜뉴스,[파이낸셜뉴스] 바이오로그디바이스가 강세다. 이 회사의 자회사가 투자한 샐바시온의 ...,"바이오로그디바스이스, 자회사 투자한 코빅실 '美FDA GRAS'.."


In [146]:
# pd_data.to_csv('./data.csv', index=False, header=True, encoding='utf-8-sig')

In [2]:
conn = sqlite3.connect('./db_scrap.stock')
c = conn.cursor()
data1 = pd.read_sql('SELECT * FROM article', con=conn)
c.close()

conn = sqlite3.connect('./db.stock')
c = conn.cursor()
data2 = pd.read_sql('SELECT * FROM article', con=conn)
c.close()

alldata = pd.concat([data1, data2])

alldata = alldata.drop_duplicates(['title'], ignore_index=True)
alldata = alldata.sort_values(by=['date','time'], axis=0, ascending=False)
completedata = alldata.reset_index(drop=True)

con = sqlite3.connect('./complete_db.stock')
c = con.cursor()
completedata.to_sql('article', con, index=False)
c.close()

In [5]:
conn = sqlite3.connect('./complete_db.stock')
c = conn.cursor()
data = pd.read_sql('SELECT * FROM article', con=conn)
c.close()

data

,id,date,time,title,content,press,stock,posi_nega
0,1,2021-08-11,15:39,북한 한미연합훈련 맹비난에 경협주 하락(종합),[그래픽] 최근 남북관계 주요 일지(서울=연합뉴스) 이재윤 기자 = 김영철 노동당 ...,연합뉴스,stock,posi_nega
1,2,2021-08-11,15:36,"원티드랩, 코스닥 상장 첫날 '따상' 성공(종합)",원티드랩[프레인키웨스트 제공. 재판매 및 DB금지](서울=연합뉴스) 이미령 기자 =...,연합뉴스,stock,posi_nega
2,3,2021-08-11,14:38,가비아 COM 도메인 다년 연장 이벤트,9월 1일부터 .com 도메인 7% 인상향후 8년에 걸쳐 인상 예정[가비아 제공][...,헤럴드경제,stock,posi_nega
3,4,2021-08-11,14:37,"한솔로지스틱스, 해상운임 급등에 순익 8배…재평가 기대",한솔로지스틱스가 강세다. 2분기 실적 개선 소식이 주가에 영향을 준 것으로 보인다....,아시아경제,stock,posi_nega
4,5,2021-08-11,14:19,"엔지켐생명과학, 경구형 코로나 치료제 호주서 상용화 첫발 소식에 강세",[파이낸셜뉴스]코로나19 4차 대유행 속 게임체인저로 꼽히는 머크의 코로나19 치료...,파이낸셜뉴스,stock,posi_nega
...,...,...,...,...,...,...,...,...
16164,7990,2019-02-27,09:14,막 오르는 2차 북미 정상회담 …경협주도 연일 들썩,[머니투데이 진경진 기자] 2차 북·미 정상회담을 맞아 남북 경협 관련 주식들이 연...,머니투데이,stock,posi_nega
16165,7991,2019-02-27,09:14,"바이로메드, 임상결과 지연 소식에 하락세",[이데일리 이광수 기자] 바이로메드(084990)가 주력 파이프라인인 VM202-D...,이데일리,stock,posi_nega
16166,7992,2019-02-27,09:13,"에이치엔티, 300억원 투자유치 소식에 상한가",[머니투데이 김사무엘 기자] 에이치엔티가 총 300억원 규모의 전환사채 발생과 유상...,머니투데이,stock,posi_nega
16167,7993,2019-02-27,09:13,"에이치엔티, 300억 CB·유증 발행 소식에 상한가",[이데일리 이명철 기자] 최대주주 변경 계약을 맺은 에이치엔티(176440)가 대규...,이데일리,stock,posi_nega
